### Vanna.AI

![vanna](img/vannaai.png)
Reference: https://vanna.ai/docs/

In [2]:
import pandas as pd
df = pd.read_csv("/Users/wangzeyu/Desktop/Github projects/legalai-chatbot/data/Students Social Media Addiction.csv")
df.head()

,Student_ID,Age,Gender,Academic_Level,Country,Avg_Daily_Usage_Hours,Most_Used_Platform,Affects_Academic_Performance,Sleep_Hours_Per_Night,Mental_Health_Score,Relationship_Status,Conflicts_Over_Social_Media,Addicted_Score
0,1,19,Female,Undergraduate,Bangladesh,5.2,Instagram,Yes,6.5,6,In Relationship,3,8
1,2,22,Male,Graduate,India,2.1,Twitter,No,7.5,8,Single,0,3
2,3,20,Female,Undergraduate,USA,6.0,TikTok,Yes,5.0,5,Complicated,4,9
3,4,18,Male,High School,UK,3.0,YouTube,No,7.0,7,Single,1,4
4,5,21,Male,Graduate,Canada,4.5,Facebook,Yes,6.0,6,In Relationship,2,7


In [3]:
import sqlite3
con = sqlite3.connect('/Users/wangzeyu/Desktop/Github projects/legalai-chatbot/rag-optimizations/chroma-sql/tmp_sql.db')
df.to_sql("media-addiction", con, if_exists='replace', index=False)

705

In [4]:
table_name = "media-addiction"
cursor = con.cursor()
# This query looks in SQLite's system table sqlite_master to find the original CREATE TABLE statement for the specified table.
query = f"SELECT sql FROM sqlite_master WHERE type='table' AND name='{table_name}'"
cursor.execute(query)
# Gets the first (and only) row from the result, then extracts the first column (the SQL statement).
create_table_sql = cursor.fetchone()[0]
print(create_table_sql)
cursor.close()
con.close()

CREATE TABLE "media-addiction" (
"Student_ID" INTEGER,
  "Age" INTEGER,
  "Gender" TEXT,
  "Academic_Level" TEXT,
  "Country" TEXT,
  "Avg_Daily_Usage_Hours" REAL,
  "Most_Used_Platform" TEXT,
  "Affects_Academic_Performance" TEXT,
  "Sleep_Hours_Per_Night" REAL,
  "Mental_Health_Score" INTEGER,
  "Relationship_Status" TEXT,
  "Conflicts_Over_Social_Media" INTEGER,
  "Addicted_Score" INTEGER
)


In [5]:
from vanna.openai import OpenAI_Chat
from vanna.chromadb import ChromaDB_VectorStore
from openai import OpenAI

In [6]:
class MyVanna(ChromaDB_VectorStore, OpenAI_Chat):
    def __init__(self, client=None, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        OpenAI_Chat.__init__(self, client=client, config=config)

client = OpenAI(base_url='http://0.0.0.0:8000/v1', api_key="na")

vn = MyVanna(client=client, config={
    'path': '/Users/wangzeyu/Desktop/Github projects/legalai-chatbot/rag-optimizations/chroma-sql', # specify the new ChromaDB location
    'temperature': 0.5, 
    'model': '/Users/wangzeyu/Desktop/Github projects/legalai-chatbot/huggingface-models/models--Qwen--Qwen2.5-0.5B-Instruct/snapshots/7ae557604adf67be50417f59c2c2f167def9a775'
})

In [7]:
vn.connect_to_sqlite('/Users/wangzeyu/Desktop/Github projects/legalai-chatbot/rag-optimizations/chroma-sql/tmp_sql.db')

In [8]:
vn.train(ddl="""
    CREATE TABLE "media-addiction" (
    "Student_ID" INTEGER,
        "Age" INTEGER,
        "Gender" TEXT,
        "Academic_Level" TEXT,
        "Country" TEXT,
        "Avg_Daily_Usage_Hours" REAL,
        "Most_Used_Platform" TEXT,
        "Affects_Academic_Performance" TEXT,
        "Sleep_Hours_Per_Night" REAL,
        "Mental_Health_Score" INTEGER,
        "Relationship_Status" TEXT,
        "Conflicts_Over_Social_Media" INTEGER,
        "Addicted_Score" INTEGER
    )
""")

Adding ddl: 
    CREATE TABLE "media-addiction" (
    "Student_ID" INTEGER,
        "Age" INTEGER,
        "Gender" TEXT,
        "Academic_Level" TEXT,
        "Country" TEXT,
        "Avg_Daily_Usage_Hours" REAL,
        "Most_Used_Platform" TEXT,
        "Affects_Academic_Performance" TEXT,
        "Sleep_Hours_Per_Night" REAL,
        "Mental_Health_Score" INTEGER,
        "Relationship_Status" TEXT,
        "Conflicts_Over_Social_Media" INTEGER,
        "Addicted_Score" INTEGER
    )



/Users/wangzeyu/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:16<00:00, 5.09MiB/s]  


'245777c4-bbc7-51c4-a7db-d7ff4df41640-ddl'

In [10]:
from vanna.flask import VannaFlaskApp
app = VannaFlaskApp(vn, allow_llm_to_see_data=True)
app.run()

Your app is running at:
http://localhost:8084
 * Serving Flask app 'vanna.flask'
 * Debug mode: on


Number of requested results 10 is greater than number of elements in index 1, updating n_results = 1
Number of requested results 10 is greater than number of elements in index 1, updating n_results = 1


Using model /Users/wangzeyu/Desktop/Github projects/legalai-chatbot/huggingface-models/models--Qwen--Qwen2.5-0.5B-Instruct/snapshots/7ae557604adf67be50417f59c2c2f167def9a775 for 391.25 tokens (approx)
Using model /Users/wangzeyu/Desktop/Github projects/legalai-chatbot/huggingface-models/models--Qwen--Qwen2.5-0.5B-Instruct/snapshots/7ae557604adf67be50417f59c2c2f167def9a775 for 266.75 tokens (approx)
